In [1]:
!pip install rouge_score

In [2]:
from datasets import load_dataset, load_metric

/Users/i0545652/article_summary/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the dataset
dataset = load_dataset('scientific_papers', 'arxiv')

/Users/i0545652/article_summary/.venv/lib/python3.8/site-packages/datasets/load.py:1429: FutureWarning: The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract', 'section_names'],
        num_rows: 6440
    })
})

In [5]:
# Get the training, validation and test datasets
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

In [6]:
train_dataset[0]["article"]

'additive models @xcite provide an important family of models for semiparametric regression or classification . some reasons for the success of additive models are their increased flexibility when compared to linear or generalized linear models and their increased interpretability when compared to fully nonparametric models .\nit is well - known that good estimators in additive models are in general less prone to the curse of high dimensionality than good estimators in fully nonparametric models .\nmany examples of such estimators belong to the large class of regularized kernel based methods over a reproducing kernel hilbert space @xmath0 , see e.g. @xcite . in the last years\nmany interesting results on learning rates of regularized kernel based models for additive models have been published when the focus is on sparsity and when the classical least squares loss function is used , see e.g. @xcite , @xcite , @xcite , @xcite , @xcite , @xcite and the references therein . of course , the

In [7]:
train_dataset[0]["abstract"]

' additive models play an important role in semiparametric statistics . \n this paper gives learning rates for regularized kernel based methods for additive models . \n these learning rates compare favourably in particular in high dimensions to recent results on optimal learning rates for purely nonparametric regularized kernel based quantile regression using the gaussian radial basis function kernel , provided the assumption of an additive model is valid . \n additionally , a concrete example is presented to show that a gaussian function depending only on one variable lies in a reproducing kernel hilbert space generated by an additive gaussian kernel , but does not belong to the reproducing kernel hilbert space generated by the multivariate gaussian kernel of the same variance .    * \n key words and phrases . * additive model , kernel , quantile regression , semiparametric , rate of convergence , support vector machine . '

In [8]:
len(train_dataset)

203037

In [9]:
from transformers import AutoTokenizer

In [10]:
model_path = "facebook/bart-large-cnn"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [12]:
max_input_length = 1024
max_output_length = 128
batch_size = 2


In [13]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=max_input_length)
    outputs = tokenizer(batch["abstract"], padding="max_length", truncation=True,
                        max_length=max_output_length)
    
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids

    # ignore the PAD token
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in
                       batch["labels"]]
    
    return batch

In [14]:
train_dataset = train_dataset.select(range(250))
val_dataset = val_dataset.select(range(25))

In [15]:
train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

In [16]:
val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

Map: 100%|██████████| 25/25 [00:00<00:00, 39.65 examples/s]


In [17]:
train_dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [18]:
train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)
val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)

In [19]:
from transformers import AutoModelForSeq2SeqLM

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_path, gradient_checkpointing=True, use_cache=False)

In [21]:
# set hyperparameters
model.config.num_beams = 2
model.config.max_length = 128
model.config.min_length = 80
model.config.length_penalty = 2.0
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3

In [22]:
rouge = load_metric("rouge")

/var/folders/yy/p134q1115s35c2fqxthfm4_c0000gn/T/ipykernel_49176/4132584981.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/Users/i0545652/article_summary/.venv/lib/python3.8/site-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [23]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids==-100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"] 
    )["rouge2"].mid 

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4)
    }

In [24]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

In [25]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #fp16=True,
    output_dir="./",
    logging_steps=5,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
)

In [26]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [27]:
trainer.train()

/Users/i0545652/article_summary/.venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 250
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8


  Gradient Accumulation steps = 4
  Total optimization steps = 31
  Number of trainable parameters = 406290432
 16%|█▌        | 5/31 [28:37<2:28:40, 343.09s/it]

{'loss': 3.5907, 'learning_rate': 4.1935483870967746e-05, 'epoch': 0.16}


 32%|███▏      | 10/31 [57:13<2:00:15, 343.60s/it]***** Running Evaluation *****
  Num examples = 25
  Batch size = 2


{'loss': 3.028, 'learning_rate': 3.387096774193548e-05, 'epoch': 0.32}


...

                                                   
 97%|█████████▋| 30/31 [22:14:03<15:02, 902.86s/it]Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoint-30/config.json


{'eval_loss': 2.7559425830841064, 'eval_rouge2_precision': 0.1337, 'eval_rouge2_recall': 0.1168, 'eval_rouge2_fmeasure': 0.1231, 'eval_runtime': 11882.112, 'eval_samples_per_second': 0.002, 'eval_steps_per_second': 0.001, 'epoch': 0.96}


Model weights saved in ./checkpoint-30/pytorch_model.bin
tokenizer config file saved in ./checkpoint-30/tokenizer_config.json
Special tokens file saved in ./checkpoint-30/special_tokens_map.json
Deleting older checkpoint [checkpoint-10] due to args.save_total_limit
100%|██████████| 31/31 [22:20:02<00:00, 4304.45s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 31/31 [22:20:02<00:00, 2593.64s/it]

{'train_runtime': 80402.8998, 'train_samples_per_second': 0.003, 'train_steps_per_second': 0.0, 'train_loss': 3.010087836173273, 'epoch': 0.99}


TrainOutput(global_step=31, training_loss=3.010087836173273, metrics={'train_runtime': 80402.8998, 'train_samples_per_second': 0.003, 'train_steps_per_second': 0.0, 'train_loss': 3.010087836173273, 'epoch': 0.99})